# Proyecto Grupal - Etapa 04: Limpieza de Datos
Autores:
Camila de la Paz
Daniel E. Ramírez
Franco Pes
Xavier Vidman

El presente archivo compila la tercera etapa del primer proyecto grupal de la carrera de Data Science de Henry, un análisis exploratorio sobre los datos proporcionados. Esta etapa se divide, a su vez, en los pasos que se detallan a continuación:
1. Importación de liberías a utilizar
2. Carga de datos
3. Limpieza y Transformación

### Paso 1: Importación de librerías

In [1]:
import datetime 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)

### Paso 2: Carga de datos


In [44]:
df = pd.read_parquet ('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2018-01.parquet')

### Paso 3: Limpieza y Transformación

#### a) Se eliminaron columnas innecesarias

In [45]:
df.drop(columns=['congestion_surcharge','airport_fee'],inplace=True)

#### b) Se eliminaron registros duplicados

In [46]:
df.drop_duplicates(inplace=True)

#### c) Se renombraron las columnas

In [47]:
df.rename(columns =
                    {'VendorID':'vendor_id',
                    'RatecodeID':'ratecode_id',
                    'PULocationID':'pu_zone_id',
                    'DOLocationID':'do_zone_id'}, inplace = True)

#### d) Se creó una nueva columna, fare_per_mile, para estudiar la relación entre fare_amount y trip_distance

In [48]:
df['fare_per_mile'] = df.fare_amount / df.trip_distance

Debido que algunos registros tenían trip_distance 0, el fare_per_mile es infinito o NaN. Se les asignó 0 a estos registros

In [49]:
df.fare_per_mile[df.fare_per_mile > 100000] = 0

C:\Users\urbda\AppData\Local\Temp\ipykernel_24704\956704063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fare_per_mile[df.fare_per_mile > 100000] = 0


In [50]:
df.fare_per_mile.fillna(0,inplace=True)

#### e) Se creó una nueva columna, trip_time, para identificar el tiempo de viaje en segundos

Primero se calculó la diferencia de tiempo

In [51]:
df['trip_time'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

Luego se convirtió a segundos

In [52]:
df.trip_time = df.trip_time.dt.total_seconds()

#### f) Se creó una nueva columna, fare_per_minute, para identificar la relación entre el fare_amount y el trip_time

In [53]:
df['fare_per_minute'] = df.fare_amount / (df.trip_time / 60)

In [54]:
df.fare_per_minute[df.fare_per_minute > 100000] = 0

C:\Users\urbda\AppData\Local\Temp\ipykernel_24704\2332422894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fare_per_minute[df.fare_per_minute > 100000] = 0


In [55]:
df.fare_per_minute.fillna(0,inplace=True)

#### g) Se creó una nueva columna, id_borough, para identificar a qué Borough pertenece cada viaje

Lo primero es cargar el dataframe con las zonas de taxis

In [56]:
df_zones = pd.read_csv('https://raw.githubusercontent.com/soyHenry/DS-Proyecto_Grupal_TaxisNYC/main/taxi%2B_zone_lookup.csv')

Se reemplazaron los nombres de los boroughs por id's

In [57]:
df_zones.Borough.replace({"Bronx":0, "Brooklyn":1, "Manhattan":2, "Queens":3, "Staten Island":4, "EWR":5, "Unknown":6}, inplace=True)


Se creó un diccionario de zonas con su respectivo borough_id para luego mapear

In [58]:
dic_zone_borough = {df_zones.LocationID[i] : df_zones.Borough[i] for i in range (0,len(df_zones))}

Se creó una nueva columna con su respectivo borough_id

In [59]:
df['borough_id'] = df.pu_zone_id.map(dic_zone_borough)

#### h) Se creó una nueva columna, id_time_borough, para posteriormente relacionar con la tabla de daots climáticos

In [60]:
df['id_time_borough'] = df.tpep_pickup_datetime.dt.strftime('%Y%m%d%H') + df.borough_id.astype(str)

#### i) Se marcaron registros que poseen outliers

Creamos la columna para identificar el outliers

In [61]:
df['outlier'] = 1

Outliers trip_distance

In [62]:
# Calculamos rango intercuartílico, mínimo, y máximo
IQR = df.trip_distance.quantile(.75) - df.trip_distance.quantile(.25)
min = df.trip_distance.quantile(.25) - (1.5 * IQR)
max = df.trip_distance.quantile(.75) + (1.5 * IQR)

# Indentificamos outliers
df.loc[df.trip_distance < min, "outlier"] = 0
df.loc[df.trip_distance > max, "outlier"] = 0

Outliers fare_amount

In [64]:
# Calculamos rango intercuartílico, mínimo, y máximo
IQR = df.fare_amount.quantile(.75) - df.fare_amount.quantile(.25)
min = df.fare_amount.quantile(.25) - (1.5 * IQR)
max = df.fare_amount.quantile(.75) + (1.5 * IQR)

# Indentificamos outliers
df.loc[df.fare_amount < min, "outlier"] = 0
df.loc[df.fare_amount > max, "outlier"] = 0

Outliers trip_time

In [65]:
# Calculamos rango intercuartílico, mínimo, y máximo
IQR = df.trip_time.quantile(.75) - df.trip_time.quantile(.25)
min = df.trip_time.quantile(.25) - (1.5 * IQR)
max = df.trip_time.quantile(.75) + (1.5 * IQR)

# Indentificamos outliers
df.loc[df.trip_time < min, "outlier"] = 0
df.loc[df.trip_time > max, "outlier"] = 0